<a href="https://colab.research.google.com/github/randaama/DS4002_Group2_Project2/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sooo we need to look at the trends of the top  10 Counties
Orange County  
Miami Dade County
Hennepin County
Hillsborough County
Broward County
Maricopa County
Hamilton County
Pinellas County
Natrona County
Fullton county
to give a desirability determination based on precipitation and average temperature. Use that to build a desiribity() function 1. Predict the average temperature and precipitation of each month based on the change in the past and then stick that into the desirability function and then rank the top Counties.

In [1]:
import pandas as pd
import plotly.express as px
#import pyearth
clim = pd.read_csv('/content/climate_data.csv')

In [2]:
import plotly.express as px
import numpy as np

In [3]:
clim.head()

,year,fips,month,temp,County_Name,State,precip
0,2003,1001,Jan,41.0,Autauga,Alabama,2.62
1,2004,1001,Jan,45.0,Autauga,Alabama,2.60
2,2005,1001,Jan,49.9,Autauga,Alabama,3.08
3,2006,1001,Jan,52.0,Autauga,Alabama,5.69
4,2007,1001,Jan,47.3,Autauga,Alabama,4.73


In [4]:
data = [
    {"Rank": 1, "County_Name": "Orange", "State": "Florida"},  # Adjusted to match the format
    {"Rank": 2, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 3, "County_Name": "Hennepin", "State": "Minnesota"},
    {"Rank": 4, "County_Name": "Hillsborough", "State": "Florida"},
    {"Rank": 5, "County_Name": "Broward", "State": "Florida"},
    {"Rank": 6, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 7, "County_Name": "Hamilton", "State": "Ohio"},
    {"Rank": 8, "County_Name": "Pinellas", "State": "Florida"},
    {"Rank": 9, "County_Name": "Natrona", "State": "Wyoming"},
    {"Rank": 10, "County_Name": "Fulton", "State": "Georgia"},
]

counties_df = pd.DataFrame(data)

# Merge climate data with counties based on 'County_Name' and 'State'
top10_wallet= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
top10_wallet['date'] = pd.to_datetime(top10_wallet['year'].astype(str) + '-' + top10_wallet['month'] + '-01')
print(top10_wallet)


      year   fips month  temp County_Name    State  precip  Rank       date
0     2003   4013   Jan  58.8    Maricopa  Arizona    0.55     6 2003-01-01
1     2004   4013   Jan  53.9    Maricopa  Arizona    0.52     6 2004-01-01
2     2005   4013   Jan  54.3    Maricopa  Arizona    2.45     6 2005-01-01
3     2006   4013   Jan  54.0    Maricopa  Arizona    0.00     6 2006-01-01
4     2007   4013   Jan  49.8    Maricopa  Arizona    0.68     6 2007-01-01
...    ...    ...   ...   ...         ...      ...     ...   ...        ...
2515  2019  56025   Dec  25.0     Natrona  Wyoming    0.51     9 2019-12-01
2516  2020  56025   Dec  25.5     Natrona  Wyoming    0.62     9 2020-12-01
2517  2021  56025   Dec  27.7     Natrona  Wyoming    0.41     9 2021-12-01
2518  2022  56025   Dec  21.0     Natrona  Wyoming    0.66     9 2022-12-01
2519  2023  56025   Dec  30.3     Natrona  Wyoming    0.14     9 2023-12-01

[2520 rows x 9 columns]


In [5]:
data = [
    {"Rank": 1, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 2, "County_Name": "Clark", "State": "Nevada"},
    {"Rank": 3, "County_Name": "Bexar", "State": "Texas"},
    {"Rank": 4, "County_Name": "Ada", "State": "Idaho"},
    {"Rank": 6, "County_Name": "New Hanover", "State": "North Carolina"},
    {"Rank": 7, "County_Name": "Collin", "State": "Texas"},
    {"Rank": 8, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 9, "County_Name": "Duval", "State": "Florida"},
    {"Rank": 10, "County_Name": "Wake", "State": "North Carolina"}
]
counties_df = pd.DataFrame(data)

# Merge climate data with counties based on 'County_Name' and 'State'
top10_migration= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
print(top10_migration)


      year   fips month  temp County_Name    State  precip  Rank
0     2003   4013   Jan  58.8    Maricopa  Arizona    0.55     1
1     2004   4013   Jan  53.9    Maricopa  Arizona    0.52     1
2     2005   4013   Jan  54.3    Maricopa  Arizona    2.45     1
3     2006   4013   Jan  54.0    Maricopa  Arizona    0.00     1
4     2007   4013   Jan  49.8    Maricopa  Arizona    0.68     1
...    ...    ...   ...   ...         ...      ...     ...   ...
2263  2019  48085   Dec  48.3      Collin    Texas    1.34     7
2264  2020  48085   Dec  47.3      Collin    Texas    3.09     7
2265  2021  48085   Dec  59.7      Collin    Texas    0.93     7
2266  2022  48085   Dec  48.4      Collin    Texas    2.53     7
2267  2023  48085   Dec  50.1      Collin    Texas    3.41     7

[2268 rows x 8 columns]


In [6]:
import pandas as pd

def calculate_weighted_weather_pattern(top10_wallet):
    # Calculate overall averages for temperature and precipitation per month across the top 10 counties
    overall_avg = top10_wallet.groupby(['month']).agg({'temp': 'mean', 'precip': 'mean'}).reset_index()

    # Calculate weights based on rank
    top10_wallet['weight'] = top10_wallet['Rank'].max() - top10_wallet['Rank'] + 1  # Invert rank for weighting

    # Calculate weighted averages for temperature and precipitation
    weighted_totals = top10_wallet.groupby(['month']).agg(
        weighted_temp=('temp', lambda x: (x * top10_wallet.loc[x.index, 'weight']).sum() / top10_wallet.loc[x.index, 'weight'].sum()),
        weighted_precip=('precip', lambda x: (x * top10_wallet.loc[x.index, 'weight']).sum() / top10_wallet.loc[x.index, 'weight'].sum())
    ).reset_index()

    # Merge the weighted totals back with the overall averages
    result = pd.merge(overall_avg, weighted_totals, on='month', how='left')

    return result

# Example usage:
# top10_wallet = pd.read_csv('your_data.csv')  # Load your data here
# weighted_results = calculate_weighted_weather_pattern(top10_wallet)
# print(weighted_results)




In [7]:
top10_wallet_ideal = calculate_weighted_weather_pattern(top10_wallet)

top10_wallet_ideal.head()

,month,temp,precip,weighted_temp,weighted_precip
0,Apr,63.943810,2.835238,66.355411,2.825991
1,Aug,79.950476,5.775286,80.930043,6.427766
2,Dec,51.170476,2.222905,54.496190,1.978026
3,Feb,50.848095,2.146333,54.038095,1.939290
4,Jan,48.161429,2.062714,51.157316,1.840632


In [8]:
top10_migration_ideal = calculate_weighted_weather_pattern(top10_migration)

top10_migration_ideal.head()

,month,temp,precip,weighted_temp,weighted_precip
0,Apr,64.600000,2.417354,64.254130,1.734568
1,Aug,82.731746,3.792698,83.853061,2.619359
2,Dec,50.528042,2.333016,49.041788,1.834470
3,Feb,51.729101,2.184656,51.067930,1.707658
4,Jan,48.668783,1.988148,48.087075,1.660311


In [15]:
# Visualize how well the weighted averages represent the averages

fig = px.line(top10_migration_ideal, x='month', y=['temp', 'weighted_temp', 'precip', 'weighted_precip'],
              title='Monthly Temperature and Precipitation based on Migration Patterns',
              labels={'value': 'Value', 'month': 'Month'},
              markers=True)

# Show the plot
fig.show()

In [16]:
# Visualize how well the weighted averages represent the averages

fig = px.line(top10_wallet_ideal, x='month', y=['temp', 'weighted_temp', 'precip', 'weighted_precip'],
              title='Monthly Temperature and Precipitation based on wallet',
              labels={'value': 'Value', 'month': 'Month'},
              markers=True)

# Show the plot
fig.show()

In [17]:
def calculate_yearly_desirability(monthly_averages, ideal_df, w_T=0.6, w_P=0.4):

    # Merge the ideal DataFrame with monthly averages
    merged_df = pd.merge(monthly_averages, ideal_df, on='month', suffixes=('', '_ideal'))

    # Calculate differences from ideal values
    merged_df['temp_diff'] = abs(merged_df['temp'] - merged_df['weighted_temp'])
    merged_df['precip_diff'] = abs(merged_df['precip'] - merged_df['weighted_precip'])

    # Normalize differences
    max_temp_diff = merged_df['temp_diff'].max()
    max_precip_diff = merged_df['precip_diff'].max()

    merged_df['normalized_temp_diff'] = merged_df['temp_diff'] / max_temp_diff
    merged_df['normalized_precip_diff'] = merged_df['precip_diff'] / max_precip_diff

    # Calculate monthly desirability score
    merged_df['desirability_month'] = (
        (1 - merged_df['normalized_temp_diff']) * w_T +
        (1 - merged_df['normalized_precip_diff']) * w_P
    )

    # Calculate yearly desirability score by taking the mean of monthly scores per county and year
    yearly_desirability = merged_df.groupby(['County_Name', 'State', 'year']).agg({'desirability_month': 'mean'}).reset_index()
    yearly_desirability.columns = ['County_Name', 'State', 'year', 'desirability_year']
    top_yearly_desirability = yearly_desirability.sort_values(by='desirability_year', ascending=False).head(10)

    return top_yearly_desirability

In [18]:
top10_migration_2023 = top10_migration[top10_migration['year'] == 2023]
clim_2023 = clim[clim['year'] == 2023]

In [19]:
calculate_yearly_desirability(top10_migration_2023,top10_migration_ideal)

,County_Name,State,year,desirability_year
3,Collin,Texas,2023,0.879906
2,Clark,Nevada,2023,0.840476
5,Maricopa,Arizona,2023,0.793838
8,Wake,North Carolina,2023,0.784387
1,Bexar,Texas,2023,0.778515
7,New Hanover,North Carolina,2023,0.760544
4,Duval,Florida,2023,0.732269
0,Ada,Idaho,2023,0.570206
6,Miami-Dade,Florida,2023,0.489522


In [20]:
calculate_yearly_desirability(clim_2023,top10_migration_ideal)

,County_Name,State,year,desirability_year
2306,Presidio,Texas,2023,0.957824
2733,Taylor,Texas,2023,0.957667
353,Callahan,Texas,2023,0.955892
2658,Stephens,Texas,2023,0.954519
2660,Sterling,Texas,2023,0.954008
832,Eastland,Texas,2023,0.953435
2530,Scurry,Texas,2023,0.952762
2547,Shackelford,Texas,2023,0.952576
2755,Throckmorton,Texas,2023,0.952540
920,Fisher,Texas,2023,0.952511


In [21]:
top10_wallet_2023 = top10_wallet[top10_wallet['year'] == 2023]

calculate_yearly_desirability(top10_wallet_2023,top10_wallet_ideal)

,County_Name,State,year,desirability_year
9,Pinellas,Florida,2023,0.815493
1,Fulton,Georgia,2023,0.815333
8,Orange,Florida,2023,0.813335
4,Hillsborough,Florida,2023,0.810173
5,Maricopa,Arizona,2023,0.777574
2,Hamilton,Ohio,2023,0.707813
6,Miami-Dade,Florida,2023,0.702752
0,Broward,Florida,2023,0.680081
3,Hennepin,Minnesota,2023,0.583474
7,Natrona,Wyoming,2023,0.480821


In [22]:
yearly_desirability_wallet_other  = calculate_yearly_desirability(clim_2023,top10_wallet_ideal)

yearly_desirability_wallet_other

,County_Name,State,year,desirability_year
1832,McIntosh,Georgia,2023,0.944744
360,Camden,Georgia,2023,0.944416
446,Charleston,South Carolina,2023,0.944208
454,Chatham,Georgia,2023,0.943505
154,Beaufort,South Carolina,2023,0.943493
451,Charlton,Georgia,2023,0.942761
2017,Nassau,Florida,2023,0.941531
1044,Glynn,Georgia,2023,0.941261
1380,Jasper,South Carolina,2023,0.941193
113,Baker,Florida,2023,0.940755


In [23]:
calculate_yearly_desirability(clim_2023,top10_migration_ideal)

,County_Name,State,year,desirability_year
2306,Presidio,Texas,2023,0.957824
2733,Taylor,Texas,2023,0.957667
353,Callahan,Texas,2023,0.955892
2658,Stephens,Texas,2023,0.954519
2660,Sterling,Texas,2023,0.954008
832,Eastland,Texas,2023,0.953435
2530,Scurry,Texas,2023,0.952762
2547,Shackelford,Texas,2023,0.952576
2755,Throckmorton,Texas,2023,0.952540
920,Fisher,Texas,2023,0.952511


In [24]:

# Maybe contrast this wallethub (financially best ) with some other list of where people actually moved
#and look at the temperature differences between the 2 as welll as the "desired"
# Weight the scores based off of temp and precip 60/40

top10_wallet[['month','temp']].groupby('month').mean()

,temp
month,
Apr,63.943810
Aug,79.950476
Dec,51.170476
Feb,50.848095
Jan,48.161429
Jul,80.600476
Jun,77.605238
Mar,57.557143
May,71.029048


In [25]:
top10_wallet[['month','precip']].groupby('month').mean()

,precip
month,
Apr,2.835238
Aug,5.775286
Dec,2.222905
Feb,2.146333
Jan,2.062714
Jul,5.407333
Jun,5.631476
Mar,2.361857
May,3.707524


In [26]:
# Map out our own ideal temperatures

ideal_conditions = {
    'Jan': {'weighted_temp': 50.0, 'weighted_precip': 3.8},
    'Feb': {'weighted_temp': 52.5, 'weighted_precip': 4.1},
    'Mar': {'weighted_temp': 55.0, 'weighted_precip': 5.2},
    'Apr': {'weighted_temp': 62.5, 'weighted_precip': 5.0},
    'May': {'weighted_temp': 70.0, 'weighted_precip': 4.0},
    'Jun': {'weighted_temp': 80.0, 'weighted_precip': 2.3},
    'Jul': {'weighted_temp': 85.0, 'weighted_precip': 3.7},
    'Aug': {'weighted_temp': 82.5, 'weighted_precip': 2.5},
    'Sep': {'weighted_temp': 70.0, 'weighted_precip': 4.0},
    'Oct': {'weighted_temp': 62.5, 'weighted_precip': 4.5},
    'Nov': {'weighted_temp': 55.0, 'weighted_precip': 4.2},
    'Dec': {'weighted_temp': 55.0, 'weighted_precip': 3.8},
}

ideal_conditions_df = pd.DataFrame.from_dict(ideal_conditions, orient='index')

# Rename the index to represent months clearly
ideal_conditions_df.index.name = 'month'

# Display the DataFrame
print(ideal_conditions_df)


       weighted_temp  weighted_precip
month                                
Jan             50.0              3.8
Feb             52.5              4.1
Mar             55.0              5.2
Apr             62.5              5.0
May             70.0              4.0
Jun             80.0              2.3
Jul             85.0              3.7
Aug             82.5              2.5
Sep             70.0              4.0
Oct             62.5              4.5
Nov             55.0              4.2
Dec             55.0              3.8


In [27]:
calculate_yearly_desirability(clim_2023,ideal_conditions_df)

,County_Name,State,year,desirability_year
600,Columbia,Arkansas,2023,0.935430
1374,Jasper,Georgia,2023,0.933293
349,Calhoun,Mississippi,2023,0.932991
1123,Grenada,Mississippi,2023,0.932786
2031,Nevada,Arkansas,2023,0.932727
2301,Prairie,Arkansas,2023,0.932616
1834,McIntosh,Oklahoma,2023,0.931395
515,Clark,Arkansas,2023,0.931279
3081,Yalobusha,Mississippi,2023,0.931123
2593,Smith,Texas,2023,0.931030


In [48]:
# Get the names of the best cities predicted in each group and label them according the group
ideal = calculate_yearly_desirability(clim_2023,ideal_conditions_df)[['County_Name','State']]
ideal['group'] = 'ideal'

migration = calculate_yearly_desirability(clim_2023,top10_migration_ideal)[['County_Name','State']]
migration['group'] = 'migration'

wallet = yearly_desirability_wallet_other[['County_Name','State']]
wallet['group'] = 'wallet'

# put all the names into one data set
all_tops = pd.concat([ideal,migration,wallet])

tops_info = pd.merge(all_tops,clim_2023,on=['County_Name','State'],how='inner')
tops_info.head()

<ipython-input-48-43159dfdbf07>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,County_Name,State,group,year,fips,month,temp,precip
0,Columbia,Arkansas,ideal,2023,5027,Jan,50.1,7.49
1,Columbia,Arkansas,ideal,2023,5027,Feb,52.2,4.01
2,Columbia,Arkansas,ideal,2023,5027,Mar,56.7,4.75
3,Columbia,Arkansas,ideal,2023,5027,Apr,61.9,7.71
4,Columbia,Arkansas,ideal,2023,5027,May,71.1,4.42


In [58]:
# Get average values by month and group, format months to be in order on graph
top_avgs = tops_info.copy()
top_avgs['month'] = pd.to_datetime(top_avgs['month'], format='%b').dt.month
top_avgs.head()

top_avgs = top_avgs.groupby(['month','group'])[['precip','temp']].mean().reset_index()
top_avgs.head()

# Create a line plot for temperature
fig_temp = px.line(
    top_avgs,
    x='month',
    y='temp',
    color='group',
    markers=True,
    title='Temperature in Top 10 Places to Retire across Groups (2023)'
)

# Update layout for temperature
fig_temp.update_layout(
    xaxis_title='Month',
    yaxis_title='Temperature (°F)',
    legend_title='Group',
)

# Label months
fig_temp.update_xaxes(
    tickvals = np.arange(1, 13),
    ticktext=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
)
# Show temperature plot
fig_temp.show()

# Create a line plot for precipitation
fig_precip = px.line(
    top_avgs,
    x='month',
    y='precip',
    color='group',
    markers=True,
    title='Precipitation in Top 10 Places to Retire across Groups (2023)'
)

# Update layout for precipitation
fig_precip.update_layout(
    xaxis_title='Month',
    yaxis_title='Precipitation (mm)',
    legend_title='Group',
)

# Label months
fig_precip.update_xaxes(
    tickvals = np.arange(1, 13),
    ticktext=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
)

# Show precipitation plot
fig_precip.show()